In [7]:
%env CYPHER=/usr/bin/cypher-shell
%env HOST=connection_uri
%env USER=neo4j
%env PW=your_password

env: CYPHER=/usr/bin/cypher-shell
env: HOST=connection_uri
env: USER=neo4j
env: PW=your_password


In [2]:
CONNECT="$CYPHER -a $HOST -u $USER -p $PW"

In [3]:
!{CONNECT} "show users"

+------------------------------------------------------------------+
| user    | roles      | passwordChangeRequired | suspended | home |
+------------------------------------------------------------------+
| "neo4j" | ["PUBLIC"] | FALSE                  | FALSE     | NULL |
+------------------------------------------------------------------+

1 row
ready to start consuming query after 237 ms, results consumed after another 668 ms


In [4]:
!{CONNECT} "MATCH (n) RETURN n LIMIT 5"

+--------------------------------------------------------------------------------------+
| n                                                                                    |
+--------------------------------------------------------------------------------------+
| (:Movie {tagline: "Welcome to the Real World", title: "The Matrix", released: 1999}) |
| (:Person {name: "Keanu Reeves", born: 1964})                                         |
| (:Person {name: "Carrie-Anne Moss", born: 1967})                                     |
| (:Person {name: "Laurence Fishburne", born: 1961})                                   |
| (:Person {name: "Hugo Weaving", born: 1960})                                         |
+--------------------------------------------------------------------------------------+

5 rows
ready to start consuming query after 75 ms, results consumed after another 12 ms


In [6]:
!{CONNECT} "MATCH (p:Person)-[:ACTED_IN]->(m:Movie) \
WHERE m.title = 'Top Gun' \
RETURN p"

+-------------------------------------------------+
| p                                               |
+-------------------------------------------------+
| (:Person {name: "Val Kilmer", born: 1959})      |
| (:Person {name: "Meg Ryan", born: 1961})        |
| (:Person {name: "Tom Skerritt", born: 1933})    |
| (:Person {name: "Kelly McGillis", born: 1957})  |
| (:Person {name: "Tom Cruise", born: 1962})      |
| (:Person {name: "Anthony Edwards", born: 1962}) |
+-------------------------------------------------+

6 rows
ready to start consuming query after 80 ms, results consumed after another 3 ms


##### Prompt 1: Which persons reviewed a movie? Return the persons and movies they reviewed. Order by person's name. 

##### Hint: the relationship type is called `REVIEWED`

##### Prompt 2: Which persons reviewed a movie and also follow the same movie's actors? Return the person's name and movie title.

##### Hint: the relationship type is called `FOLLOWS`